### 제주도 한달살기, 한달살이 - 블로그 데이터 확인

제주도 한달살기를 하는 여행객들의 패턴을 확인하기 위해 <br>
네이버 블로그에서 한달살기하는 여행객들의 후기를 살펴보고, <br>
어떤 패턴을 갖는지 학인하고자 크롤링 실시

### https://search.naver.com/

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

# 대기시간 설정을 위한 
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from urllib.parse import quote
import time
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
from datetime import datetime, timedelta
from tqdm import tqdm

# 시스템 확인 - 파일 경로 확인
import os
# 다운로드 파일이동
import shutil

#엑셀파일 불러들이기 위한
import xlrd

#time.sleep을 위한 난수생성
import random

import pandas as pd

In [1]:
### 네이버 블로그 작성일자를 날짜로 변환하는 함수
# ex 1주전 -> 7일 빼는

def convert_date(s):
    if len(s) == 11 : # 2023.12.13.  
        return s[:-1] # 마지막 점 제거
    
    now = datetime.now()
    if '어제' in s:
        date = now - timedelta(days = 1)
    elif '일' in s:
        date = now - timedelta(days = int(s[0]))
    elif '시간' in s:
        date = now - timedelta(hours = int(s[:-4]))
    elif '분' in s:
        date = now - timedelta(minutes = int(s[:-3]))
    elif '주' in s:
        date = now - timedelta(weeks = int(s[:-3]))

    return date.strftime('%Y.%m.%d')

In [6]:
# 한달살기 검색결과

base_url = 'https://search.naver.com/search.naver?where=blog&&sm=tab_jum&query='

keyword_list = ['제주도', '한달살기']
keyword_search = ''

for word in keyword_list:    
    keyword_search += f'+{quote(word)}'

# 제외하고자 하는 검색어
# 파트너스 - -%ED%8C%8C%ED%8A%B8%EB%84%88%EC%8A%A4
# 예약코드 - -%EC%98%88%EC%95%BD%EC%BD%94%EB%93%9C
delete_words = ["파트너스", "예약코드", '부동산', '이벤트', '리브애니웨어', '미스터멘션', '탁송', '퇴실청소']
delete_words

delete_search = ""
for word in delete_words:    
    delete_search += f'+-{quote(word)}'

# 검색 URL - 네이버 VIEW 중에서 BLOG / 숙소이름 / 제외검색어
url = base_url + keyword_search + delete_search
driver = webdriver.Chrome()
driver.get(url)

round(random.uniform(1.5, 2.5), 3)

while True:
    last_height = driver.execute_script("return document.body.scrollHeight")

    # 창의 내리기 끝까지 하지않고, 중간에 멈춰서 한번은 page_down을 통해서
    driver.execute_script(f'window.scrollTo(0, {last_height - random.randint(1, 100)})')
    time.sleep(round(random.uniform(0.0, 1.0), 3))
    driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.PAGE_DOWN)     
    time.sleep(round(random.uniform(1.0, 1.5), 3))

    # 창의 높이가 달라지지 않으면 스크롤 내리기 중단
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break

round(random.uniform(4.5, 6.5), 3)

soup = BeautifulSoup(driver.page_source, 'html.parser')
lis = soup.select_one('.lst_view').select('.view_wrap')

result = []

for li in lis:
    try :
        title = li.select_one('.title_area').select_one('a').text.strip()
        #title = re.sub('[-=+,/\?:^.@*\"※~ㆍ!』><‘|\(\)\[\]`\'…》\”\“\’·]', "", title_raw)
    except:
        title = 'error'
    
    try :
        content = li.select_one('.dsc_link').text.strip()
        #content = re.sub('[-=+,/\?:^.@*\"※~ㆍ!』><‘|\(\)\[\]`\'…》\”\“\’·]', "", content_raw)
    except :
        content = 'error'
    try :          
        blog_link = li.select_one('.title_area').select_one('a')['href']        
    except :
        blog_link = 'error'

    try :          
        naver_time = li.select_one('.sub').text.strip()
        blog_time = convert_date(naver_time)
    except :
        blog_time = 'error'

    if blog_link.find('blog.naver.com') > 0:
                            
        driver = webdriver.Chrome()
        driver.get(blog_link)
        driver.switch_to.frame('mainFrame')
        time.sleep(round(random.uniform(0.0, 1.2), 3))

        try : 
            lis = driver.find_element(By.CLASS_NAME, 'se-main-container').find_elements(By.CLASS_NAME, 'se-text')
        except : 
            try :
                lis = driver.find_element(By.ID, 'postListBody').find_elements(By.CLASS_NAME, 'se_textarea')
            except :
                pass

        
        
        blog_contents = []     

        for li in lis:
            #sub_content = re.sub('[-=+,#/\?:^.@*\"※~ㆍ!』><‘|\(\)\[\]`\'…》\”\“\’·]', "", li.text.strip())

            try : 
                sub_content = li.text.strip()
                blog_contents.append(sub_content.replace('\U0001f979','').replace('\n',''))
                blog_text = ' '.join(blog_contents)
            except :
                blog_text = 'error'

        time.sleep(round(random.uniform(0.5, 1.5), 3))

   
    result.append({'title' : title,
                'contents' : content,
                'time' : blog_time,
                'blog_text' : blog_text})


In [8]:
result = pd.DataFrame(result)
result

,title,contents,time,blog_text
0,돌아기랑 제주도한달살기 24일째(406일) : 비양도,첨에 제주도 한달살기 올때도 어디갈지 생각을 안하고 와서 매번 전날이나 아침에 오늘...,2023.11.27,첨에 제주도 한달살기 올때도 어디갈지 생각을 안하고 와서 매번 전날이나 아침에 오늘...
1,"제주도 한달살기 (20) - 우도 코코나라, 문빵구, 돌담집, 몬딱...",한달살기 중 두번째 우도 ㅎㅎ 처음은 전기차를 빌렸지만 이번엔 카트형식으로 나란히 ...,2023.11.07,한달살기 중 두번째 우도 ㅎㅎ처음은 전기차를 빌렸지만 이번엔 카트형식으로 나란히 앉...
2,제주동쪽숙소 제주도한달살기하기 좋은 구좌 제주오션뷰2차♪,#제주오션뷰2차 #제주도펜션 #제주동쪽숙소 제주에는 크게 동쪽과 서쪽으로 나뉘면서 ...,2023.12.13,#제주오션뷰2차#제주도펜션#제주동쪽숙소제주에는 크게 동쪽과 서쪽으로 나뉘면서 다양한...
3,제주도 한달살기 비용 (한달살이 생활비),제주도 한달살기는 돈이 적게 들지는 않지만 우리 가족에게 꽤 많은 추억을 남겼으므로...,2022.02.23,"2021년 4월에 제주도 한달살기를 하고 왔다.세식구 기준(어른 둘, 아기 하나)으..."
4,제주도 한달살기 숙소 비용 제주 중문 애견동반 더맨션 이이공공,그중에서도 제일 마음에 들었던 건 오후 시간 깊숙이 들어오는 가을 햇살 창밖에 보이...,2023.10.14,이번에는 좀 여유로운 여행을 하고 싶어서다소 긴 일정으로 제주도에 방문했어요.여행...
...,...,...,...,...
1045,제주도 한달살기 비용 절약하며 행복지수 높이기 좋다 좋아,제가 제주도 한달살기 해보는 게 소원이었는데 그걸 이루기 위함 이었습니다 한달살기를...,2023.07.11,저 정말 아주아주 행복하게 잘 지내고 있습니다이제 7월 20일 이면 서울로 돌아갑니...
1046,[제주도한달살기] 아이들이 정말 좋아하는 하루종일 놀 수 있는...,29 제주도 한달살기 애들이 가고싶다고 했던 뽀로로테마파크 가기전날 네이버로 예약했...,2022.02.22,저 정말 아주아주 행복하게 잘 지내고 있습니다이제 7월 20일 이면 서울로 돌아갑니...
1047,"제주도 한달살기 +11 - 제주김만복, 산도위치 (feat.등산준비)","제주도 한달살기 11일차 제주김만복, 산도위치 한라산 등산준비 후후.. 다음날 한라...",2021.07.22,"21.5.8.제주도 한달살기 11일차제주김만복, 산도위치한라산 등산준비후후.. 다음..."
1048,# 제주도 한달살기 _ 산방산 맛집 <춘미향식당>,"제주도 한달살기 산방산 맛집 춘미향식당 아이들과 오후내내 바다에서 물놀이하고, 언니...",2017.11.24,# 제주도 한달살기 _ 산방산 맛집 <춘미향식당> 제주도 한달살기산방산 맛집춘미향식...


In [10]:
result.to_csv('./data/jeje_month1.csv', index=False)

In [11]:
pd.read_csv('./data/jeje_month1.csv')

,title,contents,time,blog_text
0,돌아기랑 제주도한달살기 24일째(406일) : 비양도,첨에 제주도 한달살기 올때도 어디갈지 생각을 안하고 와서 매번 전날이나 아침에 오늘...,2023.11.27,첨에 제주도 한달살기 올때도 어디갈지 생각을 안하고 와서 매번 전날이나 아침에 오늘...
1,"제주도 한달살기 (20) - 우도 코코나라, 문빵구, 돌담집, 몬딱...",한달살기 중 두번째 우도 ㅎㅎ 처음은 전기차를 빌렸지만 이번엔 카트형식으로 나란히 ...,2023.11.07,한달살기 중 두번째 우도 ㅎㅎ처음은 전기차를 빌렸지만 이번엔 카트형식으로 나란히 앉...
2,제주동쪽숙소 제주도한달살기하기 좋은 구좌 제주오션뷰2차♪,#제주오션뷰2차 #제주도펜션 #제주동쪽숙소 제주에는 크게 동쪽과 서쪽으로 나뉘면서 ...,2023.12.13,#제주오션뷰2차#제주도펜션#제주동쪽숙소제주에는 크게 동쪽과 서쪽으로 나뉘면서 다양한...
3,제주도 한달살기 비용 (한달살이 생활비),제주도 한달살기는 돈이 적게 들지는 않지만 우리 가족에게 꽤 많은 추억을 남겼으므로...,2022.02.23,"2021년 4월에 제주도 한달살기를 하고 왔다.세식구 기준(어른 둘, 아기 하나)으..."
4,제주도 한달살기 숙소 비용 제주 중문 애견동반 더맨션 이이공공,그중에서도 제일 마음에 들었던 건 오후 시간 깊숙이 들어오는 가을 햇살 창밖에 보이...,2023.10.14,이번에는 좀 여유로운 여행을 하고 싶어서다소 긴 일정으로 제주도에 방문했어요.여행...
...,...,...,...,...
1045,제주도 한달살기 비용 절약하며 행복지수 높이기 좋다 좋아,제가 제주도 한달살기 해보는 게 소원이었는데 그걸 이루기 위함 이었습니다 한달살기를...,2023.07.11,저 정말 아주아주 행복하게 잘 지내고 있습니다이제 7월 20일 이면 서울로 돌아갑니...
1046,[제주도한달살기] 아이들이 정말 좋아하는 하루종일 놀 수 있는...,29 제주도 한달살기 애들이 가고싶다고 했던 뽀로로테마파크 가기전날 네이버로 예약했...,2022.02.22,저 정말 아주아주 행복하게 잘 지내고 있습니다이제 7월 20일 이면 서울로 돌아갑니...
1047,"제주도 한달살기 +11 - 제주김만복, 산도위치 (feat.등산준비)","제주도 한달살기 11일차 제주김만복, 산도위치 한라산 등산준비 후후.. 다음날 한라...",2021.07.22,"21.5.8.제주도 한달살기 11일차제주김만복, 산도위치한라산 등산준비후후.. 다음..."
1048,# 제주도 한달살기 _ 산방산 맛집 <춘미향식당>,"제주도 한달살기 산방산 맛집 춘미향식당 아이들과 오후내내 바다에서 물놀이하고, 언니...",2017.11.24,# 제주도 한달살기 _ 산방산 맛집 <춘미향식당> 제주도 한달살기산방산 맛집춘미향식...


In [ ]:
#한달살이 검색결과
base_url = 'https://search.naver.com/search.naver?where=blog&&sm=tab_jum&query='

keyword_list = ['제주도', '한달살이']
keyword_search = ''

for word in keyword_list:    
    keyword_search += f'+{quote(word)}'

# 제외하고자 하는 검색어
# 파트너스 - -%ED%8C%8C%ED%8A%B8%EB%84%88%EC%8A%A4
# 예약코드 - -%EC%98%88%EC%95%BD%EC%BD%94%EB%93%9C
delete_words = ["파트너스", "예약코드", '부동산', '이벤트', '리브애니웨어', '미스터멘션', '탁송', '퇴실청소']
delete_words

delete_search = ""
for word in delete_words:    
    delete_search += f'+-{quote(word)}'

# 검색 URL - 네이버 VIEW 중에서 BLOG / 숙소이름 / 제외검색어
url = base_url + keyword_search + delete_search
driver = webdriver.Chrome()
driver.get(url)

round(random.uniform(1.5, 2.5), 3)

while True:
    last_height = driver.execute_script("return document.body.scrollHeight")

    # 창의 내리기 끝까지 하지않고, 중간에 멈춰서 한번은 page_down을 통해서
    driver.execute_script(f'window.scrollTo(0, {last_height - random.randint(1, 100)})')
    time.sleep(round(random.uniform(0.0, 1.0), 3))
    driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.PAGE_DOWN)     
    time.sleep(round(random.uniform(1.0, 1.5), 3))

    # 창의 높이가 달라지지 않으면 스크롤 내리기 중단
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break

round(random.uniform(4.5, 6.5), 3)

soup = BeautifulSoup(driver.page_source, 'html.parser')
lis = soup.select_one('.lst_view').select('.view_wrap')

result2 = []

for li in lis:
    try :
        title = li.select_one('.title_area').select_one('a').text.strip()
        #title = re.sub('[-=+,/\?:^.@*\"※~ㆍ!』><‘|\(\)\[\]`\'…》\”\“\’·]', "", title_raw)
    except:
        title = 'error'
    
    try :
        content = li.select_one('.dsc_link').text.strip()
        #content = re.sub('[-=+,/\?:^.@*\"※~ㆍ!』><‘|\(\)\[\]`\'…》\”\“\’·]', "", content_raw)
    except :
        content = 'error'
    try :          
        blog_link = li.select_one('.title_area').select_one('a')['href']        
    except :
        blog_link = 'error'

    try :          
        naver_time = li.select_one('.sub').text.strip()
        blog_time = convert_date(naver_time)
    except :
        blog_time = 'error'

    if blog_link.find('blog.naver.com') > 0:
                            
        driver = webdriver.Chrome()
        driver.get(blog_link)
        driver.switch_to.frame('mainFrame')
        time.sleep(round(random.uniform(0.0, 1.2), 3))

        try : 
            lis = driver.find_element(By.CLASS_NAME, 'se-main-container').find_elements(By.CLASS_NAME, 'se-text')
        except : 
            try :
                lis = driver.find_element(By.ID, 'postListBody').find_elements(By.CLASS_NAME, 'se_textarea')
            except :
                pass

        
        
        blog_contents = []     

        for li in lis:
            #sub_content = re.sub('[-=+,#/\?:^.@*\"※~ㆍ!』><‘|\(\)\[\]`\'…》\”\“\’·]', "", li.text.strip())

            try : 
                sub_content = li.text.strip()
                blog_contents.append(sub_content.replace('\U0001f979','').replace('\n',''))
                blog_text = ' '.join(blog_contents)
            except :
                blog_text = 'error'

        time.sleep(round(random.uniform(0.5, 1.5), 3))

   
    result2.append({'title' : title,
                'contents' : content,
                'time' : blog_time,
                'blog_text' : blog_text})
    
result2 = pd.DataFrame(result2)
result2
result2.to_csv('./data/jeje_month2.csv', index=False)